# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_path = "../WeatherPy/Output/cities.csv"
cities = pd.read_csv(cities_path)
cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Nikolskoye,59.7035,30.7861,17.11,96,87,1.63,RU,1657396788
1,1,Kapaa,22.0752,-159.3190,26.99,87,100,2.68,US,1657396788
2,2,Zhuhai,22.2769,113.5678,29.36,94,40,3.09,CN,1657396788
3,3,Busselton,-33.6500,115.3333,13.33,59,92,8.04,AU,1657396788
4,4,Severo-Kuril'sk,50.6789,156.1250,13.43,88,100,4.85,RU,1657396788


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities[["Lat", "Lng"]].astype(float)
humidity = cities["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
cities_narrow = cities.loc[(cities["Max Temp"] > 30) & 
                  (cities["Wind Speed"] < 4) & 
                  (cities["Humidity"] < 50) &         
                  (cities["Cloudiness"] == 0)]

cities_narrow

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
77,77,Ceres,37.5949,-120.9577,34.89,21,0,3.93,US,1657396795
256,256,Uvalde,29.3669,-99.7670,38.51,34,0,3.60,US,1657396839
283,283,Camapuã,-19.5314,-54.0439,32.36,23,0,2.50,BR,1657396845
294,294,Fes,34.0372,-4.9998,35.14,21,0,3.60,MA,1657396776
340,340,Abu Kamal,34.4506,40.9171,33.35,20,0,3.90,SY,1657396855
377,377,Cairo,30.0626,31.2497,31.42,40,0,3.09,EG,1657396172
391,391,Lake Havasu City,34.4839,-114.3224,43.03,8,0,3.60,US,1657396862
485,485,Silopi,37.2497,42.4694,31.96,19,0,1.78,TR,1657396682


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = cities_narrow.loc[:, ["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
77,Ceres,US,37.5949,-120.9577,
256,Uvalde,US,29.3669,-99.7670,
283,Camapuã,BR,-19.5314,-54.0439,
294,Fes,MA,34.0372,-4.9998,
340,Abu Kamal,SY,34.4506,40.9171,
377,Cairo,EG,30.0626,31.2497,
391,Lake Havasu City,US,34.4839,-114.3224,
485,Silopi,TR,37.2497,42.4694,


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():

    location = str(row["Lat"]) + "," + str(row["Lng"])

    params["location"] = location
    
    city = row["City"]
    print(f"Retrieving Results for Index {index}: {city}.")
    
    response = requests.get(base_url, params=params).json()
    
    results = response["results"]
    
    
    try:
        print(f"The first {city} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
              
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
    print("------------")


Retrieving Results for Index 77: Ceres.
The first Ceres hotel is Howard Johnson by Wyndham Modesto Ceres.
------------
Retrieving Results for Index 256: Uvalde.
Missing field/result... skipping.
------------
Retrieving Results for Index 283: Camapuã.
The first Camapuã hotel is Comercial Estrela.
------------
Retrieving Results for Index 294: Fes.
The first Fes hotel is Hotel Mounia Fès.
------------
Retrieving Results for Index 340: Abu Kamal.
The first Abu Kamal hotel is البوكمال.
------------
Retrieving Results for Index 377: Cairo.
The first Cairo hotel is Grand Hotel.
------------
Retrieving Results for Index 391: Lake Havasu City.
The first Lake Havasu City hotel is Heat Hotel.
------------
Retrieving Results for Index 485: Silopi.
The first Silopi hotel is Özenler Residence Otel, Şırnak Silopi.
------------


In [14]:
hotel_df_clean = hotel_df.loc[hotel_df["Hotel Name"] != "NaN"]
hotel_df_clean

,City,Country,Lat,Lng,Hotel Name
77,Ceres,US,37.5949,-120.9577,Howard Johnson by Wyndham Modesto Ceres
283,Camapuã,BR,-19.5314,-54.0439,Comercial Estrela
294,Fes,MA,34.0372,-4.9998,Hotel Mounia Fès
340,Abu Kamal,SY,34.4506,40.9171,البوكمال
377,Cairo,EG,30.0626,31.2497,Grand Hotel
391,Lake Havasu City,US,34.4839,-114.3224,Heat Hotel
485,Silopi,TR,37.2497,42.4694,"Özenler Residence Otel, Şırnak Silopi"


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]
locations = hotel_df_clean[["Lat", "Lng"]]

In [17]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))